In [1]:
import sys
import os
import math
import random
import numpy as np
import pandas as pd
import urllib
import subprocess
import re
import tempfile
import itertools
import torch
import spacy
import amrlib
import penman
from typing import List
from operator import itemgetter 
# import qgrid

In [2]:
BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')

In [3]:
from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.mdmmt_api import mdmmt_api
from nebula_api.atomic2020.comet_enrichment_api import *
from nebula_api.canonisation_api import CANON_API
from nlp_tools.light_house_generator import LightHouseGenerator

In [4]:
import nebula_api.playground_api as pg_api

In [5]:
nre = NRE_API()
api = RemoteAPIUtility()
mdmmt = mdmmt_api.MDMMT_API()
comet = Comet("/home/gil/dev/NEBULA2/nebula_api/atomic2020/comet-atomic_2020_BART")
ascore = CANON_API()
stog = amrlib.load_stog_model()
gtos = amrlib.load_gtos_model()
lh_gen = LightHouseGenerator(comet,stog,gtos)

INFO:tensorflow:Restoring parameters from /home/gil/dev/NEBULA2/nebula_api/mdmmt_api/ckpts/vggish_model.ckpt


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform

model loaded


In [6]:
movies = ['Movies/114206952',
'Movies/114207205',
'Movies/114207398',
'Movies/114207499',
'Movies/114207361',
'Movies/114207740',
'Movies/114207908',
'Movies/114208744',
'Movies/114206724',
'Movies/114206548',
'Movies/114206264']

In [7]:
from IPython.display import Javascript
from IPython.display import HTML, display
import base64


def download_video_file(movie, fname='/tmp/video_file.mp4'):    
    if os.path.exists(fname):
        os.remove(fname)
    query = 'FOR doc IN Movies FILTER doc._id == "{}" RETURN doc'.format(movie)
    cursor = api.db.aql.execute(query)
    url_prefix = "http://ec2-3-120-189-231.eu-central-1.compute.amazonaws.com:7000/"
    url_link = ''
    for doc in cursor:
        url_link = url_prefix+doc['url_path']
        url_link = url_link.replace(".avi", ".mp4")   
        print(url_link)
        urllib.request.urlretrieve(url_link, fname) 
    return fname
    # video = cv2.VideoCapture(self.temp_file)
    # fps = video.get(cv2.CAP_PROP_FPS)
    # return(fps, url_link)



def read_video_segm(abspath, t_beg, t_end):
    cmd = f'ffmpeg -y -ss {t_beg} -i {abspath} -max_muxing_queue_size 9999  -loglevel error -f mp4 -vf scale="(floor(112/ih * iw/2))*2:112"  -c:a copy  -movflags frag_keyframe+empty_moov -t {t_end - t_beg} pipe:1 -nostats -hide_banner -nostdin'
    p = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE)
    assert p.returncode == 0, cmd
    buf = p.stdout
    return buf

video_id_cnt = 0    
class VideoElem:
    def __init__(self, fname, t_start=0, t_end=999):
        with open(fname, 'rb') as f:
            #data = base64.standard_b64encode(f.read())
            buf = read_video_segm(fname, t_start, t_end)
            data = base64.standard_b64encode(buf)
        global video_id_cnt
        video_id_cnt += 1
        self.video_id_cnt = video_id_cnt
        elem = HTML(f"""
            <video id="video_{self.video_id_cnt}" autoplay loop muted>
                <source src="data:video/mp4;base64,{data.decode('ascii')}" type="video/mp4">
            </video>        
        """)
        display(elem)
    
    def hide(self):
        js = f'$("#video_{self.video_id_cnt}").hide()'
        display(Javascript(js))
        
    def show(self):
        js = f'$("#video_{self.video_id_cnt}").show()'
        display(Javascript(js))

    def remove(self):
        js = f'$("#video_{self.video_id_cnt}").remove()'
        display(Javascript(js))
        
def mdmmt_video_encode(start_f, stop_f, path='/tmp/video_file.mp4', freq=24):
        t_start = start_f//freq
        t_end = stop_f//freq
        if t_start == t_end:
            t_start = t_start - 1
        print("Start/stop", t_start, " ", t_end)
        if (t_end - t_start) >= 1:
            vemb = mdmmt.encode_video(
                mdmmt.vggish_model,  # adio modality
                mdmmt.vmz_model,  # video modality
                mdmmt.clip_model,  # image modality
                mdmmt.model_vid,  # aggregator
                path, t_start, t_end)
            return(vemb)
        else:
            print("Stage too short")
            return(None)

In [8]:
def frame_to_concepts(frame)-> List:
    def transform_concept(c):
        exp = re.compile(r"^([a-zA-z]+)(\d*)$")
        r = exp.match(c)
        return r.group(1) if r else c
        
    pre_concepts = set(frame['tracker_description']).union(set(frame['step_description'])).union(set(frame['simulated_expert']))
    concepts = list(set(map(transform_concept,pre_concepts)))
    return concepts

def kgbart_fusion(frames) -> (List[str], List[str]):
    h, outname = tempfile.mkstemp(text=True)
    os.close(h)
    h, fname = tempfile.mkstemp(text=True)
    os.close(h)
    KGBART_MAIN = BASE_DIR+'/kgbart/KGBART/KGBART_training/decode_seq2seq.py'
    KGBART_CC_DIR = BASE_DIR+'/kgbart/downloaded/commongen_dataset'
    KGBART_MODEL_DIR = BASE_DIR+'/kgbart/output/best_model/model.best.bin'
    options = {
        'data_dir': KGBART_CC_DIR,
        'output_dir': os.path.dirname(outname),
        'input_file': fname,
        'model_recover_path': KGBART_MODEL_DIR,
        'output_file': os.path.basename(outname),
        'split': 'dev',
        'beam_size': 5,
        'forbid_duplicate_ngrams': True
    }
    all_concepts = []
    with open(fname, 'w') as f:
        for frame in frames:
            concepts = frame_to_concepts(frame)
            all_concepts.append(', '.join(concepts))
            f.write(' '.join(concepts)+'\n')
        
    # write expert tokens to input file
    
    cmdline = 'python '+KGBART_MAIN+' '+ ' '.join(['--{} {}'.format(k,v) for (k,v) in options.items()]) + '>/dev/null 2>&1'
    os.system(cmdline)
    with open(outname,'r') as f:
        rc = f.readlines()
    os.unlink(outname)
    os.unlink(fname)
    return all_concepts, rc

In [18]:
def flatten(lst): return [x for l in lst for x in l]

def compute_batch_scores(image_emb: torch.Tensor, texts: List[str]) -> List[float]:
    emb_batch = mdmmt.batch_encode_text(texts)
    return (emb_image.expand_as(emb_batch)*emb_batch).sum(dim=1)


def compute_concat_score(image_emb: torch.Tensor, texts: List[str], join_on=',') -> float:
    combined_text = ""
    for t in [x.strip() for x in texts]:
        if t[-1]=='.':
            t = t[:-1]       
        t+=join_on
        t+=' '
        combined_text+=t
    print("Combined: "+combined_text)
    return torch.matmul(image_emb,mdmmt.encode_text(combined_text.strip()) )       

In [10]:
def transform_concept(c):
    exp = re.compile(r"^([a-zA-z]+)-?(\d*)$")
    r = exp.match(c)
    return r.group(1) if r else c

class ConceptManager:
    def __init__(self):
        pass
    def ground_concept(concept):
        return transform_concept(concept)

In [11]:
class SimilarityManager:
    def __init__(self):
        self.nlp = spacy.load('en_core_web_lg')

    def similarity(self, c1, c2):
        if type(c2) is not list:
            c2 = [c2]   
        a = self.nlp(c1)
        targets = self.nlp(' '.join(c2))
        return [a.similarity(x) for x in targets]


smanager = SimilarityManager()


In [12]:
class SubsetOptimization:
    def __init__(self, video_emb, experts: List, candidates_strings: List[str]):
        self.stog = amrlib.load_stog_model()
        self.video_emb = video_emb
        self.initial_temp = 90
        self.final_temp = .1
        self.alpha = 0.01
        self.theta = 0.5
        self.experts = experts
        self.candidates_strings = candidates_strings
        self.candidates_amr_strings = self.stog.parse_sents(self.candidates_strings) 
        self.candidates = self.candidates_amr_strings
        self.candidates_amrs = [penman.decode(x) for x in self.candidates_amr_strings]
        self.candidates_similarity = compute_batch_scores(self.video_emb, self.candidates_strings)                
        self.smanager = SimilarityManager()

        self.coverage_matrix = np.zeros([len(self.experts),len(self.candidates)])
        self.coverage_matrix[:] = np.nan
        # for i in range(len(experts)):
        #     for j in range(len(candidates)):
        #         self.coverage_matrix=self.concept_amr_similarity(self.experts[i],self.candidates_amrs[j])


    def concept_amr_similarity(self, concept, amr):
        insts = [ConceptManager.ground_concept(x.target) for x in amr.instances()]
        sims = self.smanager.similarity(concept, insts)
        return max(sims)

    def get_coverage(self,i,j):        
        if np.isnan(self.coverage_matrix[i][j]):
            self.coverage_matrix[i][j] = self.concept_amr_similarity(self.experts[i],self.candidates_amrs[j])
        return self.coverage_matrix[i][j]

    def get_cost(self, state: List[int]) -> float:
        # state_amrs = itemgetter(*state)(self.candidates)
        # experts_coverage = [max([self.concept_amr_similarity(x,self.candidates_amrs[i]) for i in state]) for x in self.experts]    # A list of partial coverege       
        experts_coverage = [max([self.get_coverage(i,j) for j in state]) for i in range(len(self.experts))]    # A list of partial coverege        
        coverage_score = sum(experts_coverage) / len(self.experts)
        similarity_score = self.candidates_similarity[state].mean().item()
        return coverage_score + self.theta*similarity_score

    def get_candidate(self, state: List[int]) -> List[int]:
        pass

    def get_scored_permutations(self, k):
        n = len(self.candidates)
        return [(x,self.get_cost(list(x))) for x in itertools.permutations(range(n),k)]
        
def simulated_annealing(self, initial_state):
    """Peforms simulated annealing to find a solution"""
 
    current_temp = self.initial_temp

    # Start by initializing the current state with the initial state
    current_state = initial_state
    solution = current_state

    while current_temp > self.final_temp:
        next_cand = self.get_candidate(current_state)

        # Check if next_cand is best so far
        cost_diff = get_cost(self.current_state) - get_cost(next_cand)

        # if the new solution is better, accept it
        if cost_diff > 0:
            solution = next_cand
        # if the new solution is not better, accept it with a probability of e^(-cost/temp)
        else:
            if random.uniform(0, 1) < math.exp(-cost_diff / current_temp):
                solution = next_cand
        # decrement the temperature
        current_temp -= alpha

    return solution



In [13]:
mid = movies[1]
events, places = comet.get_playground_data(mid, 0)
video = VideoElem(download_video_file(mid))
movie_info = api.get_movie_info(mid)
emb_image = mdmmt_video_encode(*movie_info['scene_elements'][0])
movie_info


0
http://ec2-3-120-189-231.eu-central-1.compute.amazonaws.com:7000/static/development/0028_The_Crying_Game_00_53_53_876-00_53_55_522.mp4


Start/stop 0   2


{'arango_id': 'Movies/114207205',
 'description': '0028_The_Crying_Game_00_53_53_876-00_53_55_522',
 'fps': 24,
 'width': 1920,
 'height': 1080,
 'last frame': 300,
 'movie_id': 'afef996f341d4d0eb5ba37085eb6c7f5',
 'mdfs': [[2, 24, 46]],
 'scene_elements': [[0, 48]]}

In [14]:
concepts, attributes, persons, triplets, verbs = lh_gen.decompose_lighthouse(events=events, actions=[],
                                                                             places=places)

concepts = flatten(concepts.values())
attributes = flatten(attributes.values())
triplets = flatten(triplets.values())
persons = flatten(persons.values())

In [ ]:
best_sent, best_res = lh_gen.generate_from_concepts(concepts, attributes, persons, triplets, verbs,
                                                    places, None)


In [ ]:
cands = comet.get_groundings(events,places,type="triplet")
for c in cands.keys():
    texts = cands[c]
    cands[c] = [x for x in texts if x.strip()]
# compute_batch_scores(embamrs = stog.parse_sents(sentences)_image,["woman hurriedly steps off of the curb into the street carrying her luggage to the car", "hurriedly steps out of the street", "hurriedly steps out of the street carrying her luggage to the car"])
# compute_batch_scores(emb_image,["man is wearing a wide-brimmed hat", "man is wearing a wide - brimmed floral hat", "man wants to take off their hat", "man with a hat"])
# compute_batch_scores(emb_image,["woman hurriedly steps off of the curb into the street carrying her luggage to the car", "woman hurriedly steps out of the street", "woman hurriedly steps out of the street carrying her luggage to the car"])
diffs = []
orig_c = []
best_c = []
all_output = []
all_scores = []
inp_scores = []
for c in cands.keys():
    # print('working on: {}'.format(c))
    final_cands = [re.sub('PersonX', 'woman', x.strip()) for x in cands[c]]
    final_c = re.sub('PersonX', 'woman', c.strip())
    orig_score = compute_batch_scores(emb_image, [final_c])
    rc = compute_batch_scores(emb_image, final_cands)
    best_score = rc.max()
    ind = rc.cpu().numpy().argmax()
    all_output.append(final_cands)
    all_scores.append(rc)
    orig_c.append(final_c)
    inp_scores.append(orig_score)
    best_c.append(final_cands[ind])
    diffs.append(rc.max()-orig_score[0])
    # print("orig score for {}: {}. Max score after: {}. Diff: {}".format(final_c,orig_score,rc.max(), rc.max()-orig_score[0]))

In [ ]:
['{} -> {} : {}'.format(x,y,z.item()) for (x,y,z) in zip(orig_c,best_c,diffs)]

In [ ]:
z1 = [x for l in all_output for x in l]
z2 = torch.concat(all_scores)
z3 = list(zip(z1,[round(x,4) for x in z2.cpu().tolist()]))
z3 = list(set(z3))
z4 = sorted(z3,key=lambda x: x[1])


In [14]:
# sentences = [x[0] for x in z4[-10:]]
sentences = ['woman in the alleyway',
             'woman alley outside apartments',
             'woman enters the narrow street',
             'woman need find a dark street corner',
             'woman the dark street corner',
             'woman need to find the alley way',
             'woman sees the friend across the street',
             'woman need to find a narrow street',
             'woman narrow street',
             'woman  the alleyway']


In [15]:
opt = SubsetOptimization(emb_image, ["woman", "street", "dark", "hurry"], sentences)

In [16]:
rc = opt.get_scored_permutations(3)

In [21]:
itemgetter(*list(reversed(sorted(rc,key = lambda x: x[1])))[0][0])(opt.candidates_strings)

('woman  the alleyway',
 'woman need to find a narrow street',
 'woman the dark street corner')

In [ ]:
[b] = stog.parse_sents(['The woman looks at a rabid dog as it bites an old, well-dressed man'])
a = penman.decode(b)

In [ ]:
concept_amr_similarity("male",a)

In [ ]:
nlp = spacy.load('en_core_web_lg')
z = nlp('dog cat milk')

In [ ]:
z[1:]

In [ ]:
gtos = amrlib.load_gtos_model()

In [ ]:
z = stog.parse_sents(['The woman looks at a rabid dog as it bites an old, well-dressed man',
                     'I was late to the airport and almost missed my flight'])
gtos.generate(z)[0]


In [ ]:
ascore.get_concept_from_entity('friendship')

In [ ]:
ConceptManager.ground_concept('bite-01')

In [ ]:
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer


In [ ]:
get_giltest('love')

In [ ]:
WordNetLemmatizer().lemmatize('chairs', 'n')